<a href="https://colab.research.google.com/github/mnewham/IOIntroduction/blob/main/ERSA_data_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analyzing market data in R 

In this notebook we will perform some exploratory analysis in R using beer sales data from a collection of retailers around Chicago. You will also learn how to do some basic data cleaning, how to merge datasets together, and how to do some basic plotting.

We will be using a version of the [Dominick’s Dataset](https://www.chicagobooth.edu/research/kilts/datasets/dominicks) available through the Kits Center for Marketing at the University of Chicago. A codebook and further documentation for the Dominick’s Dataset can be found [here](https://www.chicagobooth.edu/-/media/enterprise/centers/kilts/datasets/dominicks-dataset/dominicks-manual-and-codebook_kiltscenter.aspx).

You will need two datasets to complete these tasks. Both files are CSV’s and can be found on Google drive. 



The dataset beer.csv contains movement data for beer sales, by product, for transactions conducted across many Dominick’s stores around Chicago. Data is collected across a roughly 6-year period. This dataset is collapsed at the store-UPC-week level, meaning that each row corresponds to a given product (by UPC) for a given store (by ID) in each week. You can think of each row as giving a summary of transactions for a given product, in each store, in a given week.

* STORE: The ID of the Dominick’s store from which the data was 
collected
* CITY: The municipality in which the Dominick’s store is located
* UPC: The UPC of the product. This will be used as a key to merge with the UPC file.
* WEEK: a number representing the week in which the transactions occurred.
* QTY: The number of items which were sold in each week.
* PRICE: The price at which the product was sold.
* SALES: The total revenue of products sold. Equal to QTY * PRICE.

The dataset upc_beer.csv contains product information for each UPC in the dataset. The dataset is in collected at the UPC level, one row per product. Below are a few of the important variables.

* UPC: The UPC of the product. This will be used as a key to merge with the beer_upc dataset file.
* DESCRIP: A string representing a description of the product.
* SIZE: A string indicating the size of the case of beer. For example, 6/12O indicates 6 cans of 12 oz of beer.
* UNITS: the number of 12 oz equivalent units of beer associated with this product.
* BRAND: A string representing the brand of the Beer in question. Many products may be offered by a single brand.
* COMPANY: A string representing Beer producer’s parent company. Many brands may be owned by a single company. In turn, many companies may be owned by a single parent organization.
* TYPE: A category variable indicating the type of beer.

# Upload the data to Google collab
Start by uploading the required data files to collab. You can do this by navigating to the folder symbol on the left hand side and clicking the first icon "Upload to session storage" and selecting the relevant files. The data will now be available for the session. 


# Install the required libraries

In [8]:
library(dplyr) # Import the needed libraries, note: this make take a few minutes
install.packages('bit64')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



# Read in the data

In [9]:
beer <- fread('beer.csv') # read in the beer.csv file
head(beer) # view the top rows of the data table
str(beer)
summary(beer)
table(beer$CITY)

UPC,STORE,CITY,WEEK,QTY,PRICE,SALES
<int64>,<int>,<chr>,<int>,<int>,<dbl>,<dbl>
3410010505,2,RIVER FOREST,149,0,0,0
3410010505,2,RIVER FOREST,161,0,0,0
3410010505,2,RIVER FOREST,154,0,0,0
3410010505,2,RIVER FOREST,126,0,0,0
3410010505,2,RIVER FOREST,107,0,0,0
3410010505,2,RIVER FOREST,146,0,0,0


In [10]:
upc_beer <- fread('upc_beer.csv')
head(upc_beer)

UPC,DESCRIP,SIZE,UNITS,BRAND,COMPANY,TYPE
<int64>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>
8328000001,GUINESS STOUT,16.9 O,2,GUINNESS,DIAGEO,Stout / Porter
8382012455,GUINESS STOUT,16.9 O,2,GUINNESS,DIAGEO,Stout / Porter
1969380011,GUINESS STOUT GIFT B,8/12 O,8,GUINNESS,DIAGEO,Stout / Porter
8382012360,GUINNESS PUB DRAUGHT,4/15 O,4,GUINNESS,DIAGEO,Stout / Porter
8382012393,GUINNESS STOUT N.R.B,6/12 O,6,GUINNESS,DIAGEO,Stout / Porter
8382072553,KALIBER NON-ALCH NR,6/12 O,6,KALIBER,DIAGEO,Non-alcaholic


# Part 1 

First we will explore the data and answer the following questions:
1. What are total retail sales of beer according to this dataset? 
2. How many items were sold according to this dataset?
3. What is the average price per item sold?
4. Which store had the most sales; which store had the least? Where are these stores located?

In [15]:
# Question 1.1 calculate total sales
total.sales <- sum(beer$SALES)
total.sales

[1] 21978996

In [16]:
# Question 1.2 calculate total quantity
total.qty <- sum(beer$QTY)
total.qty

[1] 4031855

In [17]:
# Question 1.3 calculate average price per item
average.price = sum(beer$SALES) / sum(beer$QTY)
average.price

[1] 5.451336

In [35]:
# Question 1.4 Calculate sales by store

beer <- group_by(beer, STORE) # Tell dplyr which variable to aggregate by
store.sales <- summarize(beer, SALES = sum(SALES)) # Aggregate and include a firm sales as the sum of product-level sales
store.sales <- arrange(store.sales, desc(SALES)) # Sort by largest firm
head(store.sales) 
tail(store.sales)

STORE,SALES
<int>,<dbl>
103,786986.6
126,570534.6
100,567555.6
134,564091.9
128,557442.2
102,533787.4


STORE,SALES
<int>,<dbl>
140,5466.52
142,4839.24
141,4110.18
88,3223.84
146,2379.22
2,181.80


In [36]:
store.sales[which.max(store.sales$SALES),]
beer$CITY[beer$STORE==103][1]
# Store 103 (BOLINGBROOK) has the most sales

store.sales[which.min(store.sales$SALES),]
beer$CITY[beer$STORE==2][1]
# Store 2 (RIVER FOREST) has the least sales

STORE,SALES
<int>,<dbl>
103,786986.6


[1] "BOLINGBROOK"

STORE,SALES
<int>,<dbl>
2,181.8


[1] "RIVER FOREST"

# Part 2

Using the merge function, combine the two datasets by the common variable – UPC -- in order to create a unified dataset of beer sales that includes product data. Label this unified dataset beer.data. For help on how to use the merge function, type ?merge into the R console. This will bring up documentation for the function.

The beer.upc file contains variables to indicate the beer’s brand as well as the parent company of the beer producer. Additionally, each UPC is tagged with a units variable which indicates the number of 12-oz equivalent units being sold (a “12 pack” would have units == 12, a “30 rack” would have units == 30, etc.).

After the merge, you have a store-week- UPC level dataset with information about each product (by UPC). Before proceeding, you should create a total.units variable by multiplying the qty and units variables together. This creates a variable which measures how many units of beer are sold.

1. Using the total.units variable you created, what is the average price of a 12 oz unit of beer.
2. What is the most popular type of beer, by sales, observed in the sample (Light Lager, Dark Ale, Stout, etc)? What is the second most popular?
3. What are the top five most popular brands by total sales.
4. What is the highest-sales brand of beer for each type? What’s the top selling Light Lager, Dark Ale, etc.

In [37]:
beer.data = merge(upc_beer, beer, on="UPC")

In [44]:
tail(beer.data)

UPC,DESCRIP,SIZE,UNITS,BRAND,COMPANY,TYPE,STORE,CITY,WEEK,QTY,PRICE,SALES,total.units
<int64>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,<int>,<dbl>,<dbl>,<int>
3410016505,MILLER RESERVE LT LN,6/12 O,6,MILLER,MILLER-COORS,Pale Lager,51,PALOS HEIGHTS,163,1,3.49,3.49,6
3410016505,MILLER RESERVE LT LN,6/12 O,6,MILLER,MILLER-COORS,Pale Lager,51,PALOS HEIGHTS,218,0,0.00,0.00,0
3410016505,MILLER RESERVE LT LN,6/12 O,6,MILLER,MILLER-COORS,Pale Lager,53,CHICAGO,93,0,0.00,0.00,0
3410016505,MILLER RESERVE LT LN,6/12 O,6,MILLER,MILLER-COORS,Pale Lager,53,CHICAGO,94,0,0.00,0.00,0
3410016505,MILLER RESERVE LT LN,6/12 O,6,MILLER,MILLER-COORS,Pale Lager,53,CHICAGO,91,0,0.00,0.00,0
3410016505,MILLER RESERVE LT LN,6/12 O,6,MILLER,MILLER-COORS,Pale Lager,53,CHICAGO,92,0,0.00,0.00,0


In [43]:
# 2.1: Calculate average 12-oz beer price
beer.data$total.units <- beer.data$UNITS*beer.data$QTY 
sum(beer.data$SALES) / sum(beer.data$total.units)
# Average price per 12-oz unit is 0.5077 

[1] 0.5077492

In [42]:
tail(beer.data)

UPC,DESCRIP,SIZE,UNITS,BRAND,COMPANY,TYPE,STORE,CITY,WEEK,QTY,PRICE,SALES,total.units
<int64>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,<int>,<dbl>,<dbl>,<int>
3410016505,MILLER RESERVE LT LN,6/12 O,6,MILLER,MILLER-COORS,Pale Lager,51,PALOS HEIGHTS,163,1,3.49,3.49,6
3410016505,MILLER RESERVE LT LN,6/12 O,6,MILLER,MILLER-COORS,Pale Lager,51,PALOS HEIGHTS,218,0,0.00,0.00,0
3410016505,MILLER RESERVE LT LN,6/12 O,6,MILLER,MILLER-COORS,Pale Lager,53,CHICAGO,93,0,0.00,0.00,0
3410016505,MILLER RESERVE LT LN,6/12 O,6,MILLER,MILLER-COORS,Pale Lager,53,CHICAGO,94,0,0.00,0.00,0
3410016505,MILLER RESERVE LT LN,6/12 O,6,MILLER,MILLER-COORS,Pale Lager,53,CHICAGO,91,0,0.00,0.00,0
3410016505,MILLER RESERVE LT LN,6/12 O,6,MILLER,MILLER-COORS,Pale Lager,53,CHICAGO,92,0,0.00,0.00,0


In [45]:
# 2.2: Calculate the most popular type of beer

type.sales <- aggregate(beer.data$SALES, by=list(Category=beer.data$TYPE), FUN=sum)
type.sales <- type.sales[order(-type.sales$x),]
print(type.sales)

# Most popular beer is Pale Lager

         Category           x
9      Pale Lager 20726374.15
6   Non-alcaholic  1036952.94
4      Dark Lager   119302.00
3        Dark Ale    43271.24
10 Stout / Porter    19983.99
5     Malt Liquor    15692.11
8        Pale Ale     9933.88
2           Cider     6052.30
7           Other     1047.90
1       Amber Ale      385.59


In [1]:
aggregate?


ERROR: ignored